In [1]:
import pandas as pd
import numpy as np
from pathlib import Path


def load_csv_with_analysis(file_path: str, delimiter: str = ','):
    """
    Профессиональная загрузка CSV с полным анализом данных

    Args:
        file_path: путь к CSV файлу
        delimiter: разделитель (по умолчанию ',')
    """
    # Проверка файла
    if not Path(file_path).exists():
        print(f"❌ Файл {file_path} не найден")
        return None

    try:
        # Автоматическое определение кодировки и загрузка
        df = pd.read_csv(
            file_path,
            delimiter=delimiter,
            encoding='utf-8',
            skipinitialspace=True,  # Убираем пробелы после разделителя
            na_values=['', 'NA', 'N/A', 'null', 'NULL', 'NaN', 'None'],  # Доп. значения пропусков
            keep_default_na=True
        )

        print(f"✅ Файл загружен: {file_path}")
        print(f"📊 Размерность: {df.shape[0]} строк × {df.shape[1]} столбцов")
        print(f"📋 Столбцы: {'|'.join(df.columns)}")

        # Анализ пропущенных значений
        print("\n" + "=" * 50)
        print("📊 АНАЛИЗ ПРОПУЩЕННЫХ ЗНАЧЕНИЙ")
        print("=" * 50)

        missing_count = df.isnull().sum()
        missing_percent = (df.isnull().sum() / len(df)) * 100

        # Создаем DataFrame для статистики
        missing_df = pd.DataFrame({
            'Колонка': df.columns,
            'Всего строк': len(df),
            'Пропущено': missing_count.values,
            'Заполнено': len(df) - missing_count.values,
            '% пропусков': missing_percent.values,
            'Тип данных': df.dtypes.values
        })

        # Сортируем по количеству пропусков
        missing_df = missing_df.sort_values('Пропущено', ascending=False)

        # Визуализация
        for idx, row in missing_df.iterrows():
            col = row['Колонка']
            missing = row['Пропущено']
            percent = row['% пропусков']
            data_type = row['Тип данных']

            # Создаем визуальную шкалу
            bar_length = 40
            filled = int((len(df) - missing) / len(df) * bar_length)
            bar = '█' * filled + '░' * (bar_length - filled)

            status = "✅ ПОЛНОСТЬЮ ЗАПОЛНЕНО" if missing == 0 else f"⚠️  пропусков: {missing}"
            print(f"{col:25} [{bar}] {status} ({percent:.1f}%) [{data_type}]")

        # Дополнительная статистика
        print("\n" + "=" * 50)
        print("📈 ДЕТАЛЬНАЯ СТАТИСТИКА")
        print("=" * 50)

        # Колонки с пропусками
        cols_with_missing = missing_df[missing_df['Пропущено'] > 0]
        if not cols_with_missing.empty:
            print(f"\n❌ Столбцов с пропусками ({len(cols_with_missing)}):")
            for _, row in cols_with_missing.iterrows():
                print(f"  • {row['Колонка']}: {row['Пропущено']} пропусков ({row['% пропусков']:.1f}%)")

        # Полностью заполненные колонки
        cols_full = missing_df[missing_df['Пропущено'] == 0]
        if not cols_full.empty:
            print(f"\n✅ Полностью заполненные столбцы ({len(cols_full)}):")
            print(f"  • {'|'.join(cols_full['Колонка'].tolist())}")

        # Общая статистика
        total_missing = df.isnull().sum().sum()
        total_cells = df.size
        print(f"\n📊 Общая статистика:")
        print(f"  • Всего ячеек: {total_cells}")
        print(f"  • Всего пропусков: {total_missing}")
        print(f"  • Процент пропусков: {(total_missing / total_cells) * 100:.2f}%")
        print(f"  • Строк с пропусками: {df.isnull().any(axis=1).sum()}")

        return df, missing_df

    except pd.errors.EmptyDataError:
        print("❌ Файл пустой")
        return None
    except pd.errors.ParserError as e:
        print(f"❌ Ошибка парсинга CSV: {e}")
        return None
    except UnicodeDecodeError:
        print("❌ Ошибка кодировки. Попробуйте другую кодировку")
        return None
    except Exception as e:
        print(f"❌ Неожиданная ошибка: {e}")
        return None


# Использование
result = load_csv_with_analysis('DatasetSnowSales2.csv', delimiter=';')

if result:
    df, stats = result

    # Дополнительный анализ по желанию
    print("\n" + "=" * 50)
    print("🔍 ПРИМЕРЫ ДАННЫХ")
    print("=" * 50)
    print("\nПервые 5 строк:")
    print(df.head())

    print("\nПоследние 5 строк:")
    print(df.tail())

    print("\n📊 Статистика по числовым колонкам:")
    print(df.describe())

✅ Файл загружен: DatasetSnowSales2.csv
📊 Размерность: 74 строк × 6 столбцов
📋 Столбцы: Date|dayweek|sales|tempday|tempnight|precipitation

📊 АНАЛИЗ ПРОПУЩЕННЫХ ЗНАЧЕНИЙ
sales                     [███████████████████████████████████████░] ⚠️  пропусков: 1 (1.4%) [float64]
Date                      [████████████████████████████████████████] ✅ ПОЛНОСТЬЮ ЗАПОЛНЕНО (0.0%) [object]
dayweek                   [████████████████████████████████████████] ✅ ПОЛНОСТЬЮ ЗАПОЛНЕНО (0.0%) [int64]
tempday                   [████████████████████████████████████████] ✅ ПОЛНОСТЬЮ ЗАПОЛНЕНО (0.0%) [int64]
tempnight                 [████████████████████████████████████████] ✅ ПОЛНОСТЬЮ ЗАПОЛНЕНО (0.0%) [int64]
precipitation             [████████████████████████████████████████] ✅ ПОЛНОСТЬЮ ЗАПОЛНЕНО (0.0%) [object]

📈 ДЕТАЛЬНАЯ СТАТИСТИКА

❌ Столбцов с пропусками (1):
  • sales: 1 пропусков (1.4%)

✅ Полностью заполненные столбцы (5):
  • Date|dayweek|tempday|tempnight|precipitation

📊 Общая статистика:
  • 